In [13]:
import os
from dotenv import load_dotenv
load_dotenv()
import mysql.connector
import pandas as pd
# display all columns
pd.set_option('display.max_columns', None)
import gspread
import json

# Access the environment variables.
mydb_pass = os.getenv('MYDB_PASS')
google_cred = os.getenv('GOOGLE_CRED')
gc = gspread.service_account_from_dict(json.loads(google_cred.replace('"', "'").replace("'", '"')))

In [2]:
host="production.cqof4esbua2o.us-west-2.rds.amazonaws.com"
user="leadershipDashboard"
password=mydb_pass
database="ckprodv2"
# database="ck_internal_data"
port=3306

In [3]:
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password,
  database=database,
  port=port
)

mycursor = mydb.cursor()

In [20]:
query = """SELECT
  requirementsTrackerItem.`itemID` AS `itemID`,
  `requirementsTrackerItemSubmission`.`itemSubmissionID` AS `itemSubmissionID`,
  `requirementsTrackerItemSubmission`.`score` AS `score`,
  `requirementsTrackerItemSubmission`.`status` AS `status`,
  `requirementsTrackerItemSubmission`.`notes` AS `notes`,
  `requirementsTrackerItemSubmission`.`completedDate` AS `completedDate`,
  `requirementsTrackerItemSubmission`.`reviewDate` AS `reviewDate`,
  `requirementsTrackerItemSubmission`.`createdAt` AS `createdAt`,
  requirementsTrackerProgram.`name` AS `Requirements Tracker Program - Program__name`,
  requirementsTrackerProgram.`programCode` AS `Requirements Tracker Program - Program__programCode`,
  requirementsTrackerProgram.`totalProgramHours` AS `Requirements Tracker Program - Program__totalProgramHours`,
  requirementsTrackerProgram.`totalAsyncTime` AS `Requirements Tracker Program - Program__totalAsyncTime`,
  requirementsTrackerItem.`name` AS `Requirements Tracker Item - Item__name`,
  requirementsTrackerItem.`maxScore` AS `Requirements Tracker Item - Item__maxScore`,
  requirementsTrackerItem.`order` AS `Requirements Tracker Item - Item__order`,
  requirementsTrackerItem.`isRequired` AS `Requirements Tracker Item - Item__isRequired`,
  requirementsTrackerItem.`minGradeToPass` AS `Requirements Tracker Item - Item__minGradeToPass`,
  requirementsTrackerChecklist.`name` AS `Requirements Tracker Checklist - Checklist__name`,
  requirementsTrackerChecklist.`type` AS `Requirements Tracker Checklist - Checklist__type`,
  `User`.`firstName` AS `User__firstName`,
  `User`.`lastName` AS `User__lastName`,
  `User - Reviewer`.`firstName` AS `User - Reviewer__firstName`,
  `User - Reviewer`.`lastName` AS `User - Reviewer__lastName`
FROM
  `requirementsTrackerItemSubmission`
 
LEFT JOIN requirementsTrackerProgram ON `requirementsTrackerItemSubmission`.`programID` = requirementsTrackerProgram.`programID`
  LEFT JOIN `school` AS `School` ON requirementsTrackerProgram.`schoolID` = `School`.`schoolID`
  LEFT JOIN requirementsTrackerItem ON `requirementsTrackerItemSubmission`.`itemID` = requirementsTrackerItem.`itemID`
  LEFT JOIN requirementsTrackerChecklist ON requirementsTrackerItem.`checklistID` = requirementsTrackerChecklist.`checklistID`
  LEFT JOIN `user` AS `User` ON `requirementsTrackerItemSubmission`.`userID` = `User`.`userID`
  LEFT JOIN `user` AS `User - Reviewer` ON `requirementsTrackerItemSubmission`.`reviewerID` = `User - Reviewer`.`userID`
WHERE
  `School`.`name` = 'Bergen County Technical Schools'
LIMIT
  100"""

In [8]:
def pull_mysql_table(table_name):
    query = f"SELECT * FROM {table_name}"

    mycursor.execute(query)
    raw_data = mycursor.fetchall()

    field_names = [i[0] for i in mycursor.description]
    raw_df = pd.DataFrame(raw_data, columns=field_names)

    return raw_df

In [5]:
def pull_mysql(query):

    mycursor.execute(query)
    raw_data = mycursor.fetchall()

    field_names = [i[0] for i in mycursor.description]
    raw_df = pd.DataFrame(raw_data, columns=field_names)

    return raw_df

In [29]:
requirementsTrackerItem = pull_mysql_table('requirementsTrackerItem')

In [21]:
df = pull_mysql(query)

In [33]:
df.head()

,itemID,itemSubmissionID,score,status,notes,completedDate,reviewDate,createdAt,Requirements Tracker Program - Program__name,Requirements Tracker Program - Program__programCode,Requirements Tracker Program - Program__totalProgramHours,Requirements Tracker Program - Program__totalAsyncTime,Requirements Tracker Item - Item__name,Requirements Tracker Item - Item__maxScore,Requirements Tracker Item - Item__order,Requirements Tracker Item - Item__isRequired,Requirements Tracker Item - Item__minGradeToPass,Requirements Tracker Checklist - Checklist__name,Requirements Tracker Checklist - Checklist__type,User__firstName,User__lastName,User - Reviewer__firstName,User - Reviewer__lastName,maxScore
0,393462,5740068,1.0,Approved,,2023-09-12 04:00:00,2023-10-07 16:33:04,2023-09-12 17:50:46,NAVTA Veterinary Assistant Certification - Cla...,Classof24,None,0,Front Desk: Greet Clients,1.00,0,1,4,Office and Hospital Procedures,amount,Semaj,Carter,Rachel,Arena,5
1,393497,5784833,1.0,Approved,,2023-09-14 04:00:00,2023-09-15 23:45:49,2023-09-14 18:28:32,NAVTA Veterinary Assistant Certification - Cla...,Classof24,None,0,Restrain Small Animals: Place and remove small...,1.00,0,1,4,Exam Room Procedures,amount,Anahi,Salazar,Rachel,Arena,5
2,393504,5784849,1.0,Approved,,2023-09-14 04:00:00,2023-09-15 23:45:49,2023-09-14 18:28:56,NAVTA Veterinary Assistant Certification - Cla...,Classof24,None,0,"Restrain Small Animals: Restrain Birds, Rabbit...",0.00,7,0,4,Exam Room Procedures,amount,Anahi,Salazar,Rachel,Arena,5
3,393511,5784863,1.0,Approved,,2023-09-14 04:00:00,2023-09-15 23:45:49,2023-09-14 18:29:24,NAVTA Veterinary Assistant Certification - Cla...,Classof24,None,0,Basic Procedures: Trim nails,1.00,14,1,4,Exam Room Procedures,amount,Anahi,Salazar,Rachel,Arena,5
4,393504,5785053,1.0,Approved,,2023-09-14 04:00:00,2023-09-15 23:45:49,2023-09-14 18:36:10,NAVTA Veterinary Assistant Certification - Cla...,Classof24,None,0,"Restrain Small Animals: Restrain Birds, Rabbit...",0.00,7,0,4,Exam Room Procedures,amount,Adriana,Quisilema,Rachel,Arena,5


In [37]:
df['User - Reviewer__lastName'].value_counts()

User - Reviewer__lastName
Arena    100
Name: count, dtype: int64

In [32]:
df['maxScore'] = 5

df.drop(['Requirements Tracker Item - Item__maxScore'], axis=1)

In [30]:
# requirementsTrackerItem.query("maxScore < minGradeToPass")

In [24]:
requirementsTrackerItem.query('itemID == 393497')

,itemID,sisID,name,maxScore,order,description,isRequired,createdAt,updatedAt,archivedAt,checklistID,minGradeToPass,submissionGroupID
169368,393497,None,Restrain Small Animals: Place and remove small...,1.00,0,1=No understanding of material\n2=Minimal unde...,1,2023-08-23 04:04:21,2023-08-23 04:04:21,NaT,8289,4.0,None


In [26]:
requirementsTrackerItem['description'][169368]

'1=No understanding of material\n2=Minimal understanding, few requirements met\n3=Partial understanding, some requirements met\n4=Consistent understanding, all requirements met\n5=Exceptional understanding, requirements exceeded'

In [28]:
df.value_counts("Requirements Tracker Item - Item__minGradeToPass") #.query("`Requirements Tracker Item - Item__maxScore` == score")

Requirements Tracker Item - Item__minGradeToPass
4    100
Name: count, dtype: int64